In [48]:
%load_ext autoreload
%autoreload 2

In [38]:
from dataset import SyntheticDataset
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from training_data import load_training_data
import torch
from tqdm import tqdm

dataset_path = (
    r"C:\Users\Parv\Doc\HelixWorks\Basecalling\code\datasets"
    r"\synthetic\synth_dataset_large.pkl"
    )

sampling_rate = 1.0

X, y = load_training_data(
        dataset_path, column_x='Squiggle', column_y='Motifs',
        sampling_rate=sampling_rate)


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.1, random_state=1)

Index(['Squiggle', 'Bases', 'read_id', 'Motifs'], dtype='object')


In [ ]:


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.nn import CTCLoss
from sklearn.preprocessing import normalize

[autoreload of nn failed: Traceback (most recent call last):
  File "c:\Users\Parv\anaconda3\envs\pytorch_gpu\Lib\site-packages\IPython\extensions\autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "c:\Users\Parv\anaconda3\envs\pytorch_gpu\Lib\site-packages\IPython\extensions\autoreload.py", line 500, in superreload
    update_generic(old_obj, new_obj)
  File "c:\Users\Parv\anaconda3\envs\pytorch_gpu\Lib\site-packages\IPython\extensions\autoreload.py", line 397, in update_generic
    update(a, b)
  File "c:\Users\Parv\anaconda3\envs\pytorch_gpu\Lib\site-packages\IPython\extensions\autoreload.py", line 349, in update_class
    if update_generic(old_obj, new_obj):
       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Parv\anaconda3\envs\pytorch_gpu\Lib\site-packages\IPython\extensions\autoreload.py", line 397, in update_generic
    update(a, b)
  File "c:\Users\Parv\anaconda3\envs\pytorch_gpu\Lib\site-packages\IPython\extensions\autoreload.py", line

In [40]:
df = pd.read_pickle(r"C:\Users\Parv\Doc\HelixWorks\Basecalling\code\motifcaller\data\synthetic\pickled_datasets\25_2_25.pkl")

In [41]:
X = df['squiggle']
Y = df['motif_seq']

In [42]:
ctc = CTCLoss(blank=0)

In [113]:
import torch.optim as optim
from nn import MotifCaller
model = MotifCaller(n_classes=16)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [117]:
ind = 0
for x, y in zip(X, Y):
    len_x = len(x)
    x = torch.tensor(x, dtype=torch.float32).view(1, len_x)
    x = normalize(x)
    print(x.shape)
    x = torch.tensor(x, dtype=torch.float32).view(1, 1, len_x)
    y = torch.tensor(y)
    model_output = model(x)
    #print(model_output.shape)
    #sum_labels = model_output.sum(axis=1)[0].detach().numpy()
    #print(np.exp(sum_labels))
    
    n_timesteps = model_output.shape[1]
    input_lengths = torch.tensor(n_timesteps)
    label_lengths = torch.tensor(len(y))

    loss = ctc(model_output[0], y, input_lengths, label_lengths)
    print(loss.item())
    #print(loss)
    loss.backward()
    optimizer.step()
    ind += 1
    
    #print([np.exp(i) for i in ]])
    if ind == 10:
        break

(1, 8034)
13.123896598815918
(1, 3415)
1.0524003505706787
(1, 9337)
3.56681227684021
(1, 7345)
5.185147285461426
(1, 7380)
6.144494533538818
(1, 11370)
7.255798816680908
(1, 3125)
3.9470643997192383
(1, 7443)
6.6799211502075195
(1, 5454)
5.731595993041992
(1, 3386)
4.050175666809082


In [101]:
print(target_lengths)

tensor(17)


In [106]:
print(input_lengths)

tensor(50)


In [108]:
print(loss)

tensor(6.3805, grad_fn=<MeanBackward0>)


### Empirical dataset

In [1]:
dataset_path = r"C:\Users\Parv\Doc\HelixWorks\Basecalling\code\datasets\empirical\empirical_train_dataset_v6.pkl"
import pandas as pd

In [2]:
df = pd.read_pickle(dataset_path)

In [3]:
df.head()

,ONT_Barcode,HW_Address,Payload,Library_Motifs,read_id,squiggle,Motifs,Spacer_Sequence,Payload_Sequence
0,1,barcode_external01_internal01,"[[2, 3, 4, 5], [1, 2, 7, 8], [1, 4, 5, 6], [4,...","[[3, 5], [1], [1], [4], [5], [], [], []]",0804c886-cd0a-4ece-87ee-adb529974699,"[0.7250000000000001, 0.7350000000000001, 0.752...","[1, 3, 5, 1, 1, 4, 5]","[11, 3, 11, 11, 5, 11, 12, 1, 12, 13, 1, 13, 1...","[11, 2, 11, 11, 3, 11, 11, 4, 11, 11, 5, 11, 1..."
1,1,barcode_external01_internal01,"[[2, 3, 4, 5], [1, 2, 7, 8], [1, 4, 5, 6], [4,...","[[5], [1], [], [7], [7], [8], [], []]",0f041c54-7071-49d3-8ae2-7a1bf25525ab,"[0.525, 0.4845238095238096, 0.5035714285714287...","[5, 1, 1, 7, 7, 7, 8]","[11, 5, 11, 12, 1, 12, 14, 7, 14, 15, 7, 15, 1...","[11, 2, 11, 11, 3, 11, 11, 4, 11, 11, 5, 11, 1..."
2,1,barcode_external01_internal01,"[[2, 3, 4, 5], [1, 2, 7, 8], [1, 4, 5, 6], [4,...","[[], [2], [], [4], [5], [6], [5], []]",1361a5db-d135-4e98-bb49-7a53c8d72991,"[0.6675799086757991, 0.5095890410958903, 0.515...","[1, 1, 2, 7, 4, 5, 6, 5]","[12, 2, 12, 14, 4, 14, 15, 5, 15, 16, 6, 16, 1...","[11, 2, 11, 11, 3, 11, 11, 4, 11, 11, 5, 11, 1..."
3,1,barcode_external01_internal01,"[[2, 3, 4, 5], [1, 2, 7, 8], [1, 4, 5, 6], [4,...","[[4], [], [4], [6], [], [6], [], []]",1b4284d2-bee2-4a15-abab-aad861447308,"[0.6846153846153846, 0.5461538461538461, 0.549...","[1, 1, 1, 4, 4, 6, 6]","[11, 4, 11, 13, 4, 13, 14, 6, 14, 16, 6, 16]","[11, 2, 11, 11, 3, 11, 11, 4, 11, 11, 5, 11, 1..."
4,1,barcode_external01_internal01,"[[2, 3, 4, 5], [1, 2, 7, 8], [1, 4, 5, 6], [4,...","[[2], [2], [1], [4], [8], [3], [3], [5]]",1cc7b7bb-cd90-485f-b744-cf846d566675,"[0.7113163972286374, 0.6628175519630484, 0.662...","[1, 2, 2, 1, 4, 8, 3, 3, 5]","[11, 2, 11, 12, 2, 12, 13, 1, 13, 14, 4, 14, 1...","[11, 2, 11, 11, 3, 11, 11, 4, 11, 11, 5, 11, 1..."


In [4]:
df.rename(columns={"Spacer_Sequence": "motif_seq"}, inplace=True)


In [ ]:
df.to_pickle(r"C:\Users\Parv\Doc\HelixWorks\Basecalling\code\motifcaller\data\empirical\pickled_datasets\28_2_25.pkl")

In [8]:
df

,ONT_Barcode,HW_Address,Payload,Library_Motifs,read_id,squiggle,Motifs,motif_seq,Payload_Sequence
0,1,barcode_external01_internal01,"[[2, 3, 4, 5], [1, 2, 7, 8], [1, 4, 5, 6], [4,...","[[3, 5], [1], [1], [4], [5], [], [], []]",0804c886-cd0a-4ece-87ee-adb529974699,"[0.7250000000000001, 0.7350000000000001, 0.752...","[1, 3, 5, 1, 1, 4, 5]","[11, 3, 11, 11, 5, 11, 12, 1, 12, 13, 1, 13, 1...","[11, 2, 11, 11, 3, 11, 11, 4, 11, 11, 5, 11, 1..."
1,1,barcode_external01_internal01,"[[2, 3, 4, 5], [1, 2, 7, 8], [1, 4, 5, 6], [4,...","[[5], [1], [], [7], [7], [8], [], []]",0f041c54-7071-49d3-8ae2-7a1bf25525ab,"[0.525, 0.4845238095238096, 0.5035714285714287...","[5, 1, 1, 7, 7, 7, 8]","[11, 5, 11, 12, 1, 12, 14, 7, 14, 15, 7, 15, 1...","[11, 2, 11, 11, 3, 11, 11, 4, 11, 11, 5, 11, 1..."
2,1,barcode_external01_internal01,"[[2, 3, 4, 5], [1, 2, 7, 8], [1, 4, 5, 6], [4,...","[[], [2], [], [4], [5], [6], [5], []]",1361a5db-d135-4e98-bb49-7a53c8d72991,"[0.6675799086757991, 0.5095890410958903, 0.515...","[1, 1, 2, 7, 4, 5, 6, 5]","[12, 2, 12, 14, 4, 14, 15, 5, 15, 16, 6, 16, 1...","[11, 2, 11, 11, 3, 11, 11, 4, 11, 11, 5, 11, 1..."
3,1,barcode_external01_internal01,"[[2, 3, 4, 5], [1, 2, 7, 8], [1, 4, 5, 6], [4,...","[[4], [], [4], [6], [], [6], [], []]",1b4284d2-bee2-4a15-abab-aad861447308,"[0.6846153846153846, 0.5461538461538461, 0.549...","[1, 1, 1, 4, 4, 6, 6]","[11, 4, 11, 13, 4, 13, 14, 6, 14, 16, 6, 16]","[11, 2, 11, 11, 3, 11, 11, 4, 11, 11, 5, 11, 1..."
4,1,barcode_external01_internal01,"[[2, 3, 4, 5], [1, 2, 7, 8], [1, 4, 5, 6], [4,...","[[2], [2], [1], [4], [8], [3], [3], [5]]",1cc7b7bb-cd90-485f-b744-cf846d566675,"[0.7113163972286374, 0.6628175519630484, 0.662...","[1, 2, 2, 1, 4, 8, 3, 3, 5]","[11, 2, 11, 12, 2, 12, 13, 1, 13, 14, 4, 14, 1...","[11, 2, 11, 11, 3, 11, 11, 4, 11, 11, 5, 11, 1..."
...,...,...,...,...,...,...,...,...,...
23934,73,barcode_external08_internal08,"[[3, 4, 5, 7], [2, 3, 4, 5], [1, 2, 4, 8], [3,...","[[], [4], [1], [8], [6], [], [6], [7]]",e1d6beff-c03a-4bab-bfa4-a4175da41be1,"[0.5864553314121037, 0.49279538904899134, 0.52...","[8, 4, 1, 8, 6, 6, 7]","[12, 4, 12, 13, 1, 13, 14, 8, 14, 15, 6, 15, 1...","[11, 3, 11, 11, 4, 11, 11, 5, 11, 11, 7, 11, 1..."
23935,73,barcode_external08_internal08,"[[3, 4, 5, 7], [2, 3, 4, 5], [1, 2, 4, 8], [3,...","[[], [4], [2], [5], [1], [], [5], []]",ea27bf18-d3df-4b52-8f9e-98d23ce43b15,"[0.6136054421768707, 0.6054421768707482, 0.621...","[8, 8, 4, 2, 5, 1, 5]","[12, 4, 12, 13, 2, 13, 14, 5, 14, 15, 1, 15, 1...","[11, 3, 11, 11, 4, 11, 11, 5, 11, 11, 7, 11, 1..."
23936,73,barcode_external08_internal08,"[[3, 4, 5, 7], [2, 3, 4, 5], [1, 2, 4, 8], [3,...","[[3], [2], [4], [5], [3], [], [], [2]]",f566dadf-dd54-4efe-8d04-94f6dededac8,"[0.7506172839506173, 0.5851851851851851, 0.612...","[8, 3, 2, 4, 5, 3, 2]","[11, 3, 11, 12, 2, 12, 13, 4, 13, 14, 5, 14, 1...","[11, 3, 11, 11, 4, 11, 11, 5, 11, 11, 7, 11, 1..."
23937,73,barcode_external08_internal08,"[[3, 4, 5, 7], [2, 3, 4, 5], [1, 2, 4, 8], [3,...","[[4], [2], [4], [8], [], [1], [6], []]",f6e6ff66-98f2-4cca-9840-cedb55ee7f9d,"[0.6947115384615384, 0.7680288461538461, 0.728...","[8, 4, 2, 4, 8, 1, 6]","[11, 4, 11, 12, 2, 12, 13, 4, 13, 14, 8, 14, 1...","[11, 3, 11, 11, 4, 11, 11, 5, 11, 11, 7, 11, 1..."


In [13]:
t = df['motif_seq'].to_numpy().flatten()

In [19]:
import numpy as np

In [20]:
for i in t:
    print(np.unique(i))

[ 1  3  4  5 11 12 13 14 15]
[ 1  5  7  8 11 12 14 15 16]
[ 2  4  5  6 12 14 15 16 17]
[ 4  6 11 13 14 16]
[ 1  2  3  4  5  8 11 12 13 14 15 16 17 18]
[ 2  3  5  6  7 13 14 15 16 17 18]
[ 1  2  4  5  8 11 14 15 16 17]
[ 2  3  4 11 12 14]
[ 2  4  5  6  8 12 13 14 15 17]
[ 1  7  8 12 13 14 15]
[ 2  5  6  7 11 12 14 15 16 17 18]
[ 7  8 14 15]
[ 1  3  5  7  8 11 14 15 16 18]
[ 1  2  3  6 11 12 13 16 17]
[ 1  5  6  7 11 12 13 14 17 18]
[ 1  4  5 11 12 13 17 18]
[ 1  3  5 12 13 15 16 17]
[ 4  6 13]
[ 1  4  5  6  7 12 13 14 17 18]
[ 3  5  6  8 12 13 16 17]
[ 1  4  5  6  7 11 12 13 14 15 16 18]
[ 6  7 15 16 17 18]
[ 1  3  6  7  8 13 14 15 16]
[ 5  7 12 15]
[ 1  4  5  7 12 13 14 15]
[ 2  4  5  7  8 12 13 14 15 16 17 18]
[ 4  5  6  8 11 12 13 14 15]
[ 1  3  7  8 11 12 13 15 16 17]
[ 1  4  6  8 11 12 14 17 18]
[ 1 12 18]
[ 1  4  6 11 12 13 16 17]
[ 3  6  7 11 13 14 17 18]
[ 2  4  6  7  8 11 12 13 14 16 17 18]
[ 1  4  5  8 11 12 13 15 16 17 18]
[ 1  2  4  6  7  8 12 13 15 17 18]
[ 1  6  8 11 12 14